# Can Sat Rejoin Example

The can sat model was taken from the [Aerospace RL repo](https://github.com/act3-ace/aerospaceRL). It is a simple 2D model that uses applied forces to move a satellite
agent around a plane. The coordinate system is relative to a "chief" satellite, where a chaser satellite is attempting to
approach without crashing into it. For the multi-agent work, this was extended to 4 chaser satellites, that all attempt to
approach the chief satellite without crashing into the chief or one another.

The model has a 4D state space, $(x,y)$ position and $(\dot x, \dot y)$ velocities. The input space consists of two input forces,
$\mathbf{F} = (F_x, F_y)^T$. The state evolution function can be written
as

  \begin{equation}
\dot{\mathbf{x}} = \begin{bmatrix} x_2 \\
 x_3 \\
3 n^2 x_0 + 2 n x_3 + \frac{1}{m_c} F_x\\
-2 n x_2 + \frac{1}{m_c} F_y \end{bmatrix}
  \end{equation}

For a common linear representation,

  \begin{equation}
\dot{\mathbf{x}} = \begin{bmatrix} 0 && 0 && 1 && 0 \\
 0 && 0 && 0 && 1 \\
3 n^2 && 0 && 0 && 2 n \\
0 && 0 && -2 n && 0 \end{bmatrix} \mathbf{x} +
\begin{bmatrix}
0 && 0 \\
0 && 0 \\
\frac{1}{m_c} && 0 \\
0 && \frac{1}{m_c} \\
\end{bmatrix} \mathbf{F}
  \end{equation}

  
  
## Controller Design

This new, multi-agent oriented goal meant that a suitable controller should be designed. While the benchmark system is meant for
RL, a simple spring system approach was taken. The input force for each element can be accomplished in three steps

1.  Construct an undirected graph of satellites $\mathcal S$ and connections $\mathcal C$ to nearby satellites,
    $$\mathcal G = (\mathcal S, \mathcal C)$$
2.  Construct a damped spring system by assigning a spring and damping constant to each connection.
3.  Solve for the force applied to each node, and use that as a control policy for each satellite. For satellite $s_i$,
    the force $F_i$ applied to the input would be,
    
    $$\mathbf{F_i} = \sum_{s_j \in \text{neighbors}} k_p (||\mathbf r||_2 - r_l) + k_d || \operatorname{proj} (\mathbf v_r, \hat{\mathbf r})||_2 $$
    
    where $\mathbf r = {\mathbf x_{j,:2} -\mathbf  x_{i,:2}}$, $\mathbf v_r = {\mathbf x_{j, 2:} - \mathbf x_{i, 2:}}$, $r_l$ is the spring rest length,
    $k_p$ is the spring constant, and $k_d$ is the damping factor. The rest length in this case is defines the ideal final distances that
    all satellites should have to one another.

Also, this technique can be done in a centralized or distributed fashion. Notably, should all satellites be able to detect one another
symmetrically, step (1) can be accomplished per satellite by &epsilon;-ball construction. If the satellites cannot agree who is a valid connection,
the graph becomes directed, and no longer solves a spring system.





In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Configuration

In [ ]:
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/cansat_rejoin_config.toml")

In [ ]:
my_conf.config_dict['components']['can0']['config']['parameters']['idx']=1

In [ ]:
def set_sat_idx(conf, identifier, number):
    conf.config_dict['components'][identifier+str(number)]['config']['parameters']['idx']=number
    
def set_sat_state(conf, identifier, number, state):
    my_conf.config_dict['components']['can'+str(number)]['config']['topics']['states']['initial'] = state

In [ ]:
states = [[-10.0, -7.0, 0.0, 0], [-10.5, 10.5, 0.0, 0], [0.0, 10.0, -0.5, 0], [10.0, -7.0, 0.5, 0.03]]

for idx, state in enumerate(states):
    set_sat_idx(my_conf, 'can', idx)
    set_sat_state(my_conf, 'can', idx, state)

In [ ]:
from IPython.display import Image

import pathlib

plot_fname = f"pub-sub-plot.png"

# plot configuration pub/sub diagram as a file -- proj specicies a dot executbale and -Gdpi is a valid dot
# argument to change the image resolution
my_conf.plot_config(fname=pathlib.Path(plot_fname).resolve(), prog=["dot", "-Gdpi=400"])

# display written file to notebook
Image(plot_fname, height=600)

## Simulation

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 25.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
# pack states into convenient data structure
states = [np.array(trajs['can'+str(idx)].states) for idx in range(4)]

In [ ]:
# create a plot of initial and final state
fig, ax = plt.subplots(figsize=(16, 8), ncols=2, sharey=True, sharex=True)
ax[0].set_xlabel('X (m)')
ax[1].set_xlabel('X (m)')
ax[0].set_ylabel('Y (m)')
ax[0].scatter(0, 0)
ax[1].scatter(0, 0)
ax[0].text(*np.array([-1, .5]), f'chief')
ax[1].text(*np.array([-1, .5]), f'chief')
ax[0].set_title("Initial State")
ax[1].set_title("Final State")
for i in range(4):
    ax[0].text(*states[i][0, :2]+np.array([-1, .5]), f'chaser-{i}')
    ax[1].text(*states[i][-1, :2]+np.array([-1, .5]), f'chaser-{i}')
    ax[0].scatter(*states[i][0, :2])
    ax[1].scatter(*states[i][-1, :2])

## Animation

In [ ]:
# can sat animation creator
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.colors import LinearSegmentedColormap


def plot_sats(states):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlabel('X (m)', size = 12)
    ax.set_ylabel('Y (m)', size = 12)
    ax.set_xlim([-20, 20])
    ax.set_ylim([-20, 20])
    ax.set_aspect('equal')
    x_vals = []
    y_vals = []
    intensity = []
    iterations = 100

    t_vals = np.linspace(0,1, iterations)

    colors = [[0,0,1,0],[0,0,1,0.5],[0,0.2,0.4,1]]
    cmap = LinearSegmentedColormap.from_list("", colors)
    scatter = ax.scatter(x_vals,y_vals, c=[], cmap=cmap, vmin=0,vmax=1, s=5, label="Chaser Satellites")
    ax.scatter(0, 0, c='r', label="Chief Satellite")
    a_circle = plt.Circle((0, 0), 3.0, color='k', fill=False)
    ax.add_artist(a_circle)
    ax.legend()

    def get_new_vals(i):
        n = np.random.randint(1,5)
        x = [s[i, 0] for s in states]
        y = [s[i, 1] for s in states]
        return list(x), list(y)

    def update(t):
        nonlocal x_vals, y_vals, intensity
        new_xvals, new_yvals = get_new_vals(t)
        x_vals.extend(new_xvals)
        y_vals.extend(new_yvals)

        scatter.set_offsets(np.c_[x_vals,y_vals])

        intensity = np.concatenate((np.array(intensity)*0.96, np.ones(len(new_xvals))))
        scatter.set_array(intensity)
        xb = [np.min(np.concatenate([s[t-100:, 0] for s in states])), 
                        np.max(np.concatenate([s[t-100:, 0] for s in states]))]
        yb = [np.min(np.concatenate([s[t-100:, 1] for s in states])), 
                        np.max(np.concatenate([s[t-100:, 1] for s in states]))]
        if t > 100:
            ax.set_xlim(xb[0] - 0.1 * np.abs(xb[0]), xb[1] + 0.1 * np.abs(xb[1]))
            ax.set_ylim(yb[0] - 0.1 * np.abs(yb[0]), yb[1] + 0.1 * np.abs(yb[1]))

        ax.set_title('Can Satellite Constellation Rejoin (Epoch: %d)' %t)

    return matplotlib.animation.FuncAnimation(fig, update, frames=1000,interval=50)

In [ ]:
%matplotlib notebook
ani = plot_sats(states)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())